<a href="https://colab.research.google.com/github/martings/Topicos_Bioinfo/blob/main/Notebook_0_FastQ_Descarga_EDA_Trimming_y_multiQC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El control de calidad (QC) de los datos de secuenciación es un paso crucial en los análisis bioinformáticos. Los archivos FastQ, que contienen las secuencias de nucleótidos y sus respectivas calidades, deben ser evaluados para asegurar que los datos son de alta calidad antes de proceder con análisis posteriores. [FastQC](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/) es una herramienta ampliamente utilizada para generar informes detallados sobre la calidad de los datos de secuenciación.

[MultiQC](https://multiqc.info/) es una herramienta que agrega múltiples informes de FastQC en un solo reporte comprensible.

In [1]:
# Instalar SRA Toolkit
!apt-get update
!apt-get install -y sra-toolkit
# Instalación de FastQC y MultiQC
!apt-get install fastqc
!pip install multiqc
#Instalacion de seqtk
!apt-get install -y seqtk
# Install fastp
!apt-get install -y fastp

# Verificar las instalaciones
!fasterq-dump --version
!seqtk 2>&1 | head -n 1
!fastqc --version
!fastp --version

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,392 kB]
Fetched 1,652 kB in 3s (483 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state informa

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  seqtk
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 30.2 kB of archives.
After this operation, 85.0 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 seqtk amd64 1.3-2 [30.2 kB]
Fetched 30.2 kB in 1s (49.7 kB/s)
Selecting previously unselected package seqtk.
(Reading database ... 122874 files and directories currently installed.)
Preparing to unpack .../archives/seqtk_1.3-2_amd64.deb ...
Unpacking seqtk (1.3-2) ...
Setting up seqtk (1.3-2) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fastp
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 193 kB of archives.
After this operation, 640 kB of 

## Repositorios de Datos Bioinformáticos

Existen numerosos repositorios de datos bioinformáticos que facilitan el almacenamiento, acceso y reutilización de datos generados en investigaciones científicas. Uno de los repositorios más importantes es el NCBI (National Center for Biotechnology Information), que proporciona acceso a una variedad de bases de datos y herramientas bioinformáticas. Entre estas bases de datos, destaca el Sequence Read Archive (SRA), que es una plataforma diseñada para almacenar secuencias de alto rendimiento generadas por diversas tecnologías de secuenciación.

El SRA es un recurso fundamental para los investigadores que necesitan acceder a datos de secuencias crudas. Los datos depositados en el SRA incluyen lecturas de secuenciación, junto con información detallada sobre las condiciones del experimento, los protocolos utilizados y la metainformación asociada. Esta información se organiza en una estructura jerárquica que incluye **BioProjects**, **Assays** y **Experiments**. Un **BioProject** agrupa proyectos relacionados y proporciona un contexto para los datos generados. Los **Assays** describen las metodologías y técnicas empleadas, mientras que los **Experiments** detallan los procedimientos específicos y las condiciones experimentales.

Para ilustrar estos conceptos, trabajaremos con datos específicos disponibles en el SRA y un BioProject en particular. Puedes acceder a la información del experimento utilizando el siguiente enlace del [SRA para SRR6853380](https://www.ncbi.nlm.nih.gov/sra/?term=SRR6853380) y obtener más detalles sobre el contexto del proyecto en el [BioProject PRJNA438545](https://www.ncbi.nlm.nih.gov/bioproject/PRJNA438545).

Estos recursos son esenciales para la investigación bioinformática, ya que permiten la replicación de estudios, la validación de resultados y el desarrollo de nuevos análisis basados en datos existentes.

In [2]:
# Crear directorio para los datos
!mkdir -p data

# Fasterq-dump es una herramienta del sra-toolkit especifica para descargar los archivos FastQ paired-end desde SRA
# Con el ID de acceso SRA que se quiere descargar (SRR6853380)
!fasterq-dump SRR6853380 --split-files --outdir data
# Comprimir los archivos FastQ para ahorrar espacio (Es una buena practica, el espacio se acaba rapido)
!gzip data/*.fastq
###Paciencia, toma un rato

spots read      : 6,900,587
reads read      : 13,801,174
reads written   : 13,801,174


### Análisis de los Archivos FastQ Originales

En esta sección, comenzamos explorando los archivos FastQ originales descargados del SRA. Primero, mostramos el tamaño de estos archivos, lo que nos da una idea de la cantidad de datos generados en el experimento.

Notamos que los archivos son bastante grandes, lo cual es típico de experimentos de secuenciación masiva como el shotgun sequencing. Este tipo de experimento genera una gran cantidad de lecturas cortas, lo que resulta en archivos de gran tamaño.

A continuación, mostramos las primeras cuatro secuencias de cada archivo FastQ. Esto nos permite observar la estructura y formato de las lecturas.

Al inspeccionar estas lecturas, podemos ver que cada lectura en el archivo FastQ consta de cuatro líneas:
1. Una línea de encabezado que empieza con `@` seguida de un identificador único.
2. La secuencia de nucleótidos.
3. Una línea que empieza con `+` que puede repetir el identificador del encabezado.
4. Una línea de calidad que corresponde a la secuencia de nucleótidos.

Finalmente, contamos el número de lecturas en cada archivo FastQ. Esto es importante para cuantificar la cobertura y la profundidad del experimento.

La cantidad de lecturas nos confirma que estamos trabajando con un experimento de secuenciación shotgun, caracterizado por generar millones de lecturas cortas para una cobertura exhaustiva del genoma o metagenoma de interés.


In [3]:
# Mostrar el tamaño de los archivos FastQ originales
!echo "Tamaño de los archivos FastQ originales:"
!ls -lh data/*.fastq.gz

# Mostrar las primeras 4 secuencias de cada archivo FastQ
!echo "Lecturas Forward"
!zcat data/SRR6853380_1.fastq.gz | head -n 16
!echo "Lecturas Reverse"
!zcat data/SRR6853380_2.fastq.gz | head -n 16

# Contar el número de lecturas en cada archivo FastQ
!echo "Número de lecturas en SRR6853380_1.fastq.gz:"
!zcat data/SRR6853380_1.fastq.gz | echo $((`wc -l`/4))

!echo "Número de lecturas en SRR6853380_2.fastq.gz:"
!zcat data/SRR6853380_2.fastq.gz | echo $((`wc -l`/4))

Tamaño de los archivos FastQ originales:
-rw-r--r-- 1 root root 468M Jun 24 02:02 data/SRR6853380_1.fastq.gz
-rw-r--r-- 1 root root 475M Jun 24 02:02 data/SRR6853380_2.fastq.gz
Lecturas Forward
@SRR6853380.1 1 length=94
GTATAATCGCACCTCCAAATAAAATTGAGGTGAAAAATGTCAAATCCCAAGCCGCTTTATATGGTGCTTGCTTATGATTCAAGTGCTCTGCCTA
+SRR6853380.1 1 length=94
BBBFFFFFF<FFFFFIFIIFIIIFFFB<FFBFIIIBFIFFFFBBFFF'7<7''7B77BFFFFBFFFFF<F'B<BF7'7<'<<'707<'<<B<BB
@SRR6853380.2 2 length=40
GTGCCGGGCATCTCCAGCAGCGGGCGCCACCCCCATTCGC
+SRR6853380.2 2 length=40
BBBFFFFFB<FFFFI7FFBBFFFF<F7'''7BF'77<700
@SRR6853380.3 3 length=45
GACCAGCGTCATCTCATGCAGCGCTTTGCCCGGCAGGTGCTTCCG
+SRR6853380.3 3 length=45
BBBFFFFFFF7BB<BB<BFF<BFFFFFFFFIIIIIII0<0''0BB
@SRR6853380.4 4 length=101
TTGCCACTGCGCCAGCGTAAAGAAGAGGTTGCTTTCGCCGCGCATCACGCGATAGGTGTTCTTGTCGAAGCTCACCACATAGCCGATCACGGCCACCATGC
+SRR6853380.4 4 length=101
BBBFFFFFFFFFFIIIIFFFFFIIIFFFFFFFFFIFBFBFBFIFFFFFFFBBFBFBBBBB<B0<BBFB70'<BBFBBBBB<<<B<BBBBBB<7<BBFBB<7
Lecturas Reverse
@SRR6853380

### Extracción de un Subconjunto de Lecturas con `seqtk`

En algunas ocasiones, es necesario trabajar con un subconjunto de datos en lugar del conjunto completo, especialmente cuando se manejan archivos de gran tamaño. Esto permite realizar análisis preliminares más rápidos y menos intensivos en recursos. Para este propósito, utilizamos `seqtk`, una herramienta eficiente para manipular archivos FastQ.

`seqtk` es una herramienta versátil que permite realizar diversas operaciones en archivos de secuencias, como filtrado, recorte y muestreo aleatorio. En este caso, utilizamos `seqtk` para extraer un subconjunto de 100,000 lecturas de los archivos FastQ originales.

Después de extraer el subconjunto, comprimimos los archivos resultantes para ahorrar espacio en disco. A continuación, mostramos el tamaño de los archivos FastQ del subconjunto y contamos el número de lecturas en cada uno de ellos para asegurarnos de que la extracción se realizó correctamente.

Este proceso de fragmentación de datos es útil en varias situaciones, como:
- Realizar pruebas rápidas antes de ejecutar análisis completos.
- Comparar diferentes métodos de análisis en un subconjunto representativo.
- Manejar limitaciones de memoria y tiempo de procesamiento en equipos con recursos limitados.

In [4]:
# Número de lecturas a extraer para el subconjunto
num_reads = 100000

# Extraer un subconjunto del archivo FastQ usando seqtk
!seqtk sample -s100 data/SRR6853380_1.fastq.gz $num_reads > data/subset_SRR6853380_1.fastq
!seqtk sample -s100 data/SRR6853380_2.fastq.gz $num_reads > data/subset_SRR6853380_2.fastq

# Comprimir el subconjunto para ahorrar espacio
!gzip data/subset_SRR6853380_1.fastq
!gzip data/subset_SRR6853380_2.fastq

# Mostrar el tamaño de los archivos FastQ del subconjunto
!echo "Tamaño de los archivos FastQ del subconjunto:"
!ls -lh data/subset_SRR6853380_*.fastq.gz

# Contar el número de lecturas en cada archivo FastQ
!echo "Número de lecturas en SRR6853380_1.fastq.gz:"
!zcat data/subset_SRR6853380_1.fastq.gz | echo $((`wc -l`/4))

!echo "Número de lecturas en SRR6853380_2.fastq.gz:"
!zcat data/subset_SRR6853380_2.fastq.gz | echo $((`wc -l`/4))


Tamaño de los archivos FastQ del subconjunto:
-rw-r--r-- 1 root root 6.6M Jun 24 02:18 data/subset_SRR6853380_1.fastq.gz
-rw-r--r-- 1 root root 6.7M Jun 24 02:18 data/subset_SRR6853380_2.fastq.gz
Número de lecturas en SRR6853380_1.fastq.gz:
100000
Número de lecturas en SRR6853380_2.fastq.gz:
100000


### Trimming de los Datos FastQ con `fastp`

El trimming es un paso crucial en el preprocesamiento de datos de secuenciación. Consiste en eliminar bases de baja calidad y adaptadores de secuencias, lo cual mejora la precisión y fiabilidad de los análisis posteriores. Esto es especialmente importante en experimentos de secuenciación masiva donde la calidad de las lecturas puede variar significativamente.

Para realizar el trimming, utilizamos `fastp`, una herramienta rápida y versátil que no solo realiza el trimming sino que también genera informes detallados sobre la calidad de los datos antes y después del proceso.

En este caso, aplicamos el trimming tanto a los datos completos como a los subconjuntos previamente generados. Esto nos permitirá comparar los efectos del trimming en ambos conjuntos de datos utilizando FastQC en análisis posteriores.

El trimming ayuda a:
- Mejorar la calidad general de las lecturas.
- Reducir el ruido en los datos, lo que puede interferir con los análisis bioinformáticos.
- Aumentar la precisión de la alineación y el ensamblaje de secuencias.

Ejecutamos `fastp` para recortar las lecturas en los archivos FastQ y guardamos los resultados en nuevos archivos comprimidos.

In [5]:
# Trim the FastQ data using fastp
!fastp -i data/subset_SRR6853380_1.fastq.gz -I data/subset_SRR6853380_2.fastq.gz -o data/subset_trimmed_SRR6853380_1.fastq.gz -O data/subset_trimmed_SRR6853380_2.fastq.gz
!fastp -i data/SRR6853380_1.fastq.gz -I data/SRR6853380_2.fastq.gz -o data/trimmed_SRR6853380_1.fastq.gz -O data/trimmed_SRR6853380_2.fastq.gz

Read1 before filtering:
total reads: 100000
total bases: 9166546
Q20 bases: 8212105(89.5878%)
Q30 bases: 6805338(74.241%)

Read2 before filtering:
total reads: 100000
total bases: 9157497
Q20 bases: 8082548(88.2615%)
Q30 bases: 6558610(71.6201%)

Read1 after filtering:
total reads: 98063
total bases: 8994921
Q20 bases: 8087550(89.9124%)
Q30 bases: 6705952(74.5527%)

Read2 aftering filtering:
total reads: 98063
total bases: 8975429
Q20 bases: 8002200(89.1567%)
Q30 bases: 6512419(72.5583%)

Filtering result:
reads passed filter: 196126
reads failed due to low quality: 3476
reads failed due to too many N: 398
reads failed due to too short: 0
reads with adapter trimmed: 118
bases trimmed due to adapters: 1145

Duplication rate: 0.792252%

Insert size peak (evaluated by paired-end reads): 141

JSON report: fastp.json
HTML report: fastp.html

fastp -i data/subset_SRR6853380_1.fastq.gz -I data/subset_SRR6853380_2.fastq.gz -o data/subset_trimmed_SRR6853380_1.fastq.gz -O data/subset_trimmed_SRR

### Análisis de Calidad con FastQC y MultiQC

Para evaluar la calidad de nuestros datos de secuenciación, utilizamos dos herramientas clave: FastQC y MultiQC.

**FastQC** es una herramienta ampliamente utilizada para realizar un control de calidad rápido de los datos de secuenciación. Genera una serie de gráficos e informes que nos permiten evaluar varios aspectos de la calidad de las lecturas, como la distribución de la calidad de las bases, el contenido GC, la presencia de adaptadores, entre otros.

**MultiQC** es una herramienta que agrega los resultados de múltiples ejecuciones de FastQC (u otras herramientas de análisis de calidad) en un único informe comprensivo. Esto es particularmente útil cuando se trabaja con muchos archivos FastQ, ya que permite una visualización y comparación centralizada de todos los datos de calidad.

En esta sección, ejecutamos FastQC en todas las muestras generadas hasta ahora (incluyendo los datos originales, los subconjuntos y los datos recortados). Luego, utilizamos MultiQC para generar un informe combinado que nos permita comparar fácilmente la calidad de todos estos conjuntos de datos.

Para una descripción detallada de los informes generados por FastQC y qué esperar de cada métrica dependiendo del tipo de experimento, se recomienda revisar la [documentación oficial de MultiQC](https://multiqc.info/), que ofrece un desglosado exhaustivo de los informes de calidad.


In [6]:
# Create directory for trimmed FastQC results
!mkdir -p QC_results

# Run FastQC on the trimmed FastQ data
!fastqc data/*.fastq.gz -o QC_results/

# List the trimmed FastQC result files
!ls QC_results/

# Ejecutar MultiQC
!multiqc QC_results/ -o QC_results/

Started analysis of SRR6853380_1.fastq.gz
Approx 5% complete for SRR6853380_1.fastq.gz
Approx 10% complete for SRR6853380_1.fastq.gz
Approx 15% complete for SRR6853380_1.fastq.gz
Approx 20% complete for SRR6853380_1.fastq.gz
Approx 25% complete for SRR6853380_1.fastq.gz
Approx 30% complete for SRR6853380_1.fastq.gz
Approx 35% complete for SRR6853380_1.fastq.gz
Approx 40% complete for SRR6853380_1.fastq.gz
Approx 45% complete for SRR6853380_1.fastq.gz
Approx 50% complete for SRR6853380_1.fastq.gz
Approx 55% complete for SRR6853380_1.fastq.gz
Approx 60% complete for SRR6853380_1.fastq.gz
Approx 65% complete for SRR6853380_1.fastq.gz
Approx 70% complete for SRR6853380_1.fastq.gz
Approx 75% complete for SRR6853380_1.fastq.gz
Approx 80% complete for SRR6853380_1.fastq.gz
Approx 85% complete for SRR6853380_1.fastq.gz
Approx 90% complete for SRR6853380_1.fastq.gz
Approx 95% complete for SRR6853380_1.fastq.gz
Analysis complete for SRR6853380_1.fastq.gz
Started analysis of SRR6853380_2.fastq.gz

### Visualización de los Resultados de MultiQC

Los resultados del análisis de calidad combinada se pueden visualizar desde cualquier navegador web. Para ello, simplemente descarga el informe generado por MultiQC. Este informe está disponible en la ruta `/content/QC_results/multiqc_report.html` en Google Colab.

Una vez descargado, abre el archivo `multiqc_report.html` en tu navegador preferido para explorar los resultados detallados del análisis de calidad de tus datos de secuenciación. Este informe incluye gráficos interactivos y secciones detalladas sobre diversos aspectos de la calidad de las lecturas, permitiéndote identificar rápidamente cualquier problema potencial y comparar la calidad entre diferentes conjuntos de datos.

### Comentarios Finales

Este notebook ha demostrado cómo realizar un análisis de control de calidad exhaustivo de datos de secuenciación utilizando herramientas como FastQC y MultiQC. Hemos cubierto varios pasos clave, incluyendo:

- La descarga y visualización de datos FastQ originales.
- La extracción de subconjuntos de datos para análisis preliminares.
- El trimming de lecturas para mejorar la calidad general de los datos.
- La ejecución de análisis de calidad y la generación de informes combinados.

Estos procedimientos son fundamentales en la bioinformática para asegurar que los datos de secuenciación sean de alta calidad y adecuados para los análisis posteriores. Al seguir estos pasos, puedes identificar y corregir problemas tempranos en tus datos, lo que mejora la fiabilidad y precisión de tus resultados de investigación.

Para más detalles y documentación adicional sobre las herramientas utilizadas, puedes visitar los siguientes enlaces:
- [FastQC](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/)
- [MultiQC](https://multiqc.info/)
- [seqtk](https://github.com/lh3/seqtk)
- [fastp](https://github.com/OpenGene/fastp)

Espero que este notebook haya sido de ayuda!

MEGS